In [13]:
import os
import json
import sqlite3
import threading
from dataclasses import dataclass
from datetime import datetime, timezone
from typing import Any, Dict, Optional, Tuple, List

import requests


# =========================
# Constants / Globals
# =========================
TROY_OZ_TO_GRAM = 31.1034768

_FX_CACHE: Dict[Tuple[str, str], float] = {}   # (TARGET, YYYY-MM-DD) -> rate
_stop_event = threading.Event()
_worker_thread: Optional[threading.Thread] = None


# =========================
# Exceptions / Models
# =========================
class ProviderError(RuntimeError):
    pass

class RateLimited(ProviderError):
    pass

@dataclass
class Quote:
    ts_utc: str
    source: str
    currency: str
    price_per_troy_oz: float
    price_per_gram: float
    fx_rate_to_target: Optional[float]
    raw_json: Dict[str, Any]


# =========================
# Configuration
# =========================
def load_config() -> Dict[str, Any]:
    """
    Reads env vars each time, so it works well in Jupyter.
    """
    db_path = os.getenv("GOLD_DB_PATH", "gold_prices.sqlite")
    target_currency = os.getenv("GOLD_CURRENCY", "USD").upper()

    providers = [p.strip().lower() for p in os.getenv(
        "GOLD_PROVIDERS",
        "gold-api,goldapi-net,metals-api,metals-dev"
    ).split(",") if p.strip()]

    cfg = {
        "DB_PATH": db_path,
        "TARGET_CURRENCY": target_currency,
        "PROVIDERS": providers,
        "REQUEST_TIMEOUT": float(os.getenv("GOLD_TIMEOUT_SEC", "20")),
        "USER_AGENT": os.getenv("GOLD_USER_AGENT", "gold-collector/1.0"),
        "GOLDAPI_NET_KEY": os.getenv("GOLDAPI_KEY", ""),
        "METALS_API_KEY": os.getenv("METALS_API_KEY", ""),
        "METALS_DEV_KEY": os.getenv("METALS_DEV_KEY", ""),
        "INTERVAL_SEC": int(os.getenv("GOLD_INTERVAL_SEC", "60")),
    }
    return cfg


# =========================
# Database (with migration)
# =========================
def ensure_db(path: str) -> None:
    with sqlite3.connect(path) as con:
        con.execute("""
            CREATE TABLE IF NOT EXISTS gold_spot_prices (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ts_utc TEXT NOT NULL,
                source TEXT NOT NULL,
                currency TEXT NOT NULL,
                price_per_troy_oz REAL NOT NULL,
                price_per_gram REAL NOT NULL,
                fx_rate_to_target REAL,
                raw_json TEXT NOT NULL
            )
        """)

        # Migration: add missing columns if you updated the script
        cols = {row[1] for row in con.execute("PRAGMA table_info(gold_spot_prices)").fetchall()}
        if "fx_rate_to_target" not in cols:
            con.execute("ALTER TABLE gold_spot_prices ADD COLUMN fx_rate_to_target REAL")

        con.execute("CREATE INDEX IF NOT EXISTS idx_gold_ts ON gold_spot_prices(ts_utc)")
        con.execute("CREATE INDEX IF NOT EXISTS idx_gold_source_cur ON gold_spot_prices(source, currency)")


def save_quote(path: str, q: Quote) -> None:
    with sqlite3.connect(path) as con:
        con.execute(
            """INSERT INTO gold_spot_prices
               (ts_utc, source, currency, price_per_troy_oz, price_per_gram, fx_rate_to_target, raw_json)
               VALUES (?, ?, ?, ?, ?, ?, ?)""",
            (
                q.ts_utc,
                q.source,
                q.currency,
                q.price_per_troy_oz,
                q.price_per_gram,
                q.fx_rate_to_target,
                json.dumps(q.raw_json, ensure_ascii=False),
            )
        )


# =========================
# HTTP helpers
# =========================
def _http_get(cfg: Dict[str, Any], url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    headers = {"User-Agent": cfg["USER_AGENT"]}
    r = requests.get(url, params=params, headers=headers, timeout=cfg["REQUEST_TIMEOUT"])

    if r.status_code == 429:
        raise RateLimited("HTTP 429 rate limited: %s" % url)
    if r.status_code in (401, 403, 402):
        raise ProviderError("HTTP %s auth/quota: %s | %s" % (r.status_code, url, r.text[:200]))

    r.raise_for_status()
    try:
        return r.json()
    except Exception as e:
        raise ProviderError("Non-JSON response from %s: %s" % (url, e))


def _looks_like_rate_limit(payload: Dict[str, Any]) -> bool:
    txt = json.dumps(payload, ensure_ascii=False).lower()
    needles = ["rate limit", "too many", "quota", "exceeded", "limit reached", "requests per", "throttle"]
    return any(n in txt for n in needles)


# =========================
# FX (only needed when converting USD->other)
# =========================
def fx_usd_to(cfg: Dict[str, Any], target: str) -> float:
    target = target.upper()
    if target == "USD":
        return 1.0

    today = datetime.now(timezone.utc).date().isoformat()
    key = (target, today)
    if key in _FX_CACHE:
        return _FX_CACHE[key]

    data = _http_get(cfg, "https://api.frankfurter.app/latest", params={"from": "USD", "to": target})
    rate = data.get("rates", {}).get(target)
    if rate is None:
        raise ProviderError("FX rate USD->%s missing in response: %s" % (target, data))

    _FX_CACHE[key] = float(rate)
    return _FX_CACHE[key]


# =========================
# Providers
# =========================
def fetch_gold_api_dot_com(cfg: Dict[str, Any], target_currency: str) -> Tuple[float, str, Dict[str, Any], Optional[float]]:
    """
    gold-api.com: typically returns XAU spot in USD per troy ounce.
    """
    data = _http_get(cfg, "https://api.gold-api.com/price/XAU")
    if _looks_like_rate_limit(data):
        raise RateLimited("Rate limit payload (gold-api.com)")

    price_usd_oz = data.get("price")
    if price_usd_oz is None:
        raise ProviderError("No 'price' field (gold-api.com): %s" % data)

    price_usd_oz = float(price_usd_oz)

    if target_currency.upper() == "USD":
        return price_usd_oz, "USD", data, None

    fx = fx_usd_to(cfg, target_currency)
    wrapped = {"source_payload": data, "fx_usd_to_target": fx, "target_currency": target_currency.upper()}
    return price_usd_oz * fx, target_currency.upper(), wrapped, fx


def fetch_goldapi_net(cfg: Dict[str, Any], target_currency: str) -> Tuple[float, str, Dict[str, Any], Optional[float]]:
    """
    goldapi.net: returns XAU in chosen currency, requires key.
    """
    if not cfg["GOLDAPI_NET_KEY"]:
        raise ProviderError("GOLDAPI_KEY missing for goldapi.net")

    url = "https://app.goldapi.net/price/XAU/%s" % target_currency.upper()
    data = _http_get(cfg, url, params={"x-api-key": cfg["GOLDAPI_NET_KEY"]})
    if _looks_like_rate_limit(data):
        raise RateLimited("Rate limit payload (goldapi.net)")

    price = data.get("price")
    if price is None:
        raise ProviderError("No 'price' field (goldapi.net): %s" % data)

    return float(price), target_currency.upper(), data, None


def fetch_metals_api(cfg: Dict[str, Any], target_currency: str) -> Tuple[float, str, Dict[str, Any], Optional[float]]:
    """
    metals-api.com: base=USD symbols=XAU; commonly rates.XAU = XAU per USD, so USD per XAU = 1/rates.XAU
    """
    if not cfg["METALS_API_KEY"]:
        raise ProviderError("METALS_API_KEY missing for metals-api.com")

    data = _http_get(
        cfg,
        "https://metals-api.com/api/latest",
        params={"access_key": cfg["METALS_API_KEY"], "base": "USD", "symbols": "XAU"}
    )

    if not data.get("success", True) and "error" in data:
        if _looks_like_rate_limit(data):
            raise RateLimited("Rate/quota payload (metals-api.com)")
        raise ProviderError("metals-api error: %s" % data.get("error"))

    xau = data.get("rates", {}).get("XAU")
    if xau is None:
        raise ProviderError("No rates.XAU (metals-api.com): %s" % data)

    usd_per_xau = 1.0 / float(xau)

    if target_currency.upper() == "USD":
        wrapped = {"source_payload": data, "note": "usd_per_xau computed as 1/rates.XAU"}
        return usd_per_xau, "USD", wrapped, None

    fx = fx_usd_to(cfg, target_currency)
    wrapped = {
        "source_payload": data,
        "fx_usd_to_target": fx,
        "target_currency": target_currency.upper(),
        "note": "usd_per_xau computed as 1/rates.XAU",
    }
    return usd_per_xau * fx, target_currency.upper(), wrapped, fx


def fetch_metals_dev(cfg: Dict[str, Any], target_currency: str) -> Tuple[float, str, Dict[str, Any], Optional[float]]:
    """
    metals.dev: requires key; response structure can vary, we try multiple field paths.
    """
    if not cfg["METALS_DEV_KEY"]:
        raise ProviderError("METALS_DEV_KEY missing for metals.dev")

    data = _http_get(
        cfg,
        "https://api.metals.dev/v1/metal/spot",
        params={"api_key": cfg["METALS_DEV_KEY"], "metal": "XAU", "currency": target_currency.upper()}
    )

    if _looks_like_rate_limit(data):
        raise RateLimited("Rate limit payload (metals.dev)")

    candidates = [
        ("price",),
        ("rate",),
        ("result", "price"),
        ("data", "price"),
        ("metal", "price"),
    ]

    def get_nested(d: Dict[str, Any], path: Tuple[str, ...]) -> Optional[Any]:
        cur: Any = d
        for k in path:
            if not isinstance(cur, dict) or k not in cur:
                return None
            cur = cur[k]
        return cur

    price = None
    for path in candidates:
        v = get_nested(data, path)
        if v is not None:
            price = v
            break

    if price is None:
        raise ProviderError("Could not find price in metals.dev response: %s" % data)

    return float(price), target_currency.upper(), data, None


# =========================
# Orchestrator / Collector
# =========================
def fetch_with_failover(cfg: Dict[str, Any]) -> Quote:
    target_currency = cfg["TARGET_CURRENCY"]
    providers = cfg["PROVIDERS"]

    ts_utc = datetime.now(timezone.utc).isoformat(timespec="seconds")
    errors: List[str] = []

    for p in providers:
        try:
            if p == "gold-api":
                price_oz, cur, raw, fx = fetch_gold_api_dot_com(cfg, target_currency)
                source = "gold-api.com"
            elif p == "goldapi-net":
                price_oz, cur, raw, fx = fetch_goldapi_net(cfg, target_currency)
                source = "goldapi.net"
            elif p == "metals-api":
                price_oz, cur, raw, fx = fetch_metals_api(cfg, target_currency)
                source = "metals-api.com"
            elif p == "metals-dev":
                price_oz, cur, raw, fx = fetch_metals_dev(cfg, target_currency)
                source = "metals.dev"
            else:
                raise ProviderError("Unknown provider: %s" % p)

            price_g = float(price_oz) / TROY_OZ_TO_GRAM
            return Quote(
                ts_utc=ts_utc,
                source=source,
                currency=cur,
                price_per_troy_oz=float(price_oz),
                price_per_gram=float(price_g),
                fx_rate_to_target=fx,
                raw_json=raw,
            )

        except RateLimited as e:
            errors.append("%s: RATE_LIMIT: %s" % (p, e))
            continue
        except (requests.RequestException, ProviderError) as e:
            errors.append("%s: ERROR: %s" % (p, e))
            continue

    raise RuntimeError("All providers failed:\n- " + "\n- ".join(errors))


def collect_once() -> Quote:
    cfg = load_config()
    ensure_db(cfg["DB_PATH"])
    q = fetch_with_failover(cfg)
    save_quote(cfg["DB_PATH"], q)
    print("OK %s | %s | %s %.2f/oz | %.2f/g | DB=%s" %
          (q.ts_utc, q.source, q.currency, q.price_per_troy_oz, q.price_per_gram, cfg["DB_PATH"]))
    return q


# =========================
# Periodic runner (Jupyter-friendly)
# =========================
def start_collector(interval_sec: Optional[int] = None) -> None:
    """
    Runs in a background thread (recommended for Jupyter).
    Stop via stop_collector().
    """
    global _worker_thread
    cfg = load_config()

    if interval_sec is None:
        interval_sec = int(cfg["INTERVAL_SEC"])

    if _worker_thread is not None and _worker_thread.is_alive():
        print("[collector] Already running")
        return

    _stop_event.clear()

    def worker():
        while not _stop_event.is_set():
            try:
                collect_once()
            except Exception as e:
                print("[collector] ERROR:", e)
            _stop_event.wait(interval_sec)

    _worker_thread = threading.Thread(target=worker, daemon=True)
    _worker_thread.start()
    print("[collector] Started (interval=%ss, currency=%s, providers=%s)" %
          (interval_sec, cfg["TARGET_CURRENCY"], ",".join(cfg["PROVIDERS"])))


def stop_collector() -> None:
    _stop_event.set()
    print("[collector] Stop signal sent")


# =========================
# Optional: blocking loop (non-thread)
# =========================
def run_forever(interval_sec: Optional[int] = None) -> None:
    """
    Blocking loop (useful outside Jupyter). Stop with KeyboardInterrupt.
    """
    cfg = load_config()
    if interval_sec is None:
        interval_sec = int(cfg["INTERVAL_SEC"])

    print("[collector] Running forever (interval=%ss). Ctrl+C to stop." % interval_sec)
    while True:
        collect_once()
        threading.Event().wait(interval_sec)


# =========================
# Defaults for your use-case (USD)
# =========================
# You can set these in Jupyter before calling start_collector()/collect_once()
# os.environ["GOLD_CURRENCY"] = "USD"
# os.environ["GOLD_PROVIDERS"] = "gold-api,goldapi-net,metals-api,metals-dev"
# os.environ["GOLD_DB_PATH"] = "gold_prices.sqlite"
# os.environ["GOLD_INTERVAL_SEC"] = "60"


In [14]:
import os
os.environ["GOLD_CURRENCY"] = "USD"
os.environ["GOLD_PROVIDERS"] = "gold-api,goldapi-net,metals-api,metals-dev"
os.environ["GOLD_DB_PATH"] = "gold_prices.sqlite"
os.environ["GOLD_INTERVAL_SEC"] = "60"  # elke minuut

In [ ]:
start_collector()  # pakt GOLD_INTERVAL_SEC


[collector] Already running


OK 2025-12-12T14:23:27+00:00 | gold-api.com | USD 4341.80/oz | 139.59/g | DB=gold_prices.sqlite
OK 2025-12-12T14:24:11+00:00 | gold-api.com | USD 4342.40/oz | 139.61/g | DB=gold_prices.sqlite
OK 2025-12-12T14:24:27+00:00 | gold-api.com | USD 4342.40/oz | 139.61/g | DB=gold_prices.sqlite
OK 2025-12-12T14:25:11+00:00 | gold-api.com | USD 4340.70/oz | 139.56/g | DB=gold_prices.sqlite
OK 2025-12-12T14:25:28+00:00 | gold-api.com | USD 4340.70/oz | 139.56/g | DB=gold_prices.sqlite
OK 2025-12-12T14:26:12+00:00 | gold-api.com | USD 4340.70/oz | 139.56/g | DB=gold_prices.sqlite


In [16]:
stop_collector()


[collector] Stop signal sent
